# Knowledge Representation

This notebook serves as the supporting material for Chapter 12 **Knowledge Representation**. In this notebook we will show how to use the code repository to represenr tge most important aspects of the real world in first order logic, such as action, space, time, thoughts and shopping. In this notebook we will be building representations of various complex domains in first order logic. First, let's load the aima-jar.

In [48]:
%classpath add jar ../out/artifacts/aima_core_jar/aima-core.jar

## Categories and objects

There are two choices for representing categories in first order logic, predicates and objects.That is, we can use the predicate $ Basketball (b)$ , or we can reify the category as an object, Basketballs. We could then say $Member (b, Basketballs )$, which we will abbreviate as $ b \in Basketballs $ , to say that b is a member of the category of basketballs. We say $Subset(Basketballs, Balls)$, abbreviated as $ Basketballs \subset Balls $, to say that Basketballs is a subcategory of Balls.

First-order logic makes it easy to state facts about categories, either by relating objects to categories or by quantifying over their members. Here are some types of facts, with
examples of each:
1. An object is a member of a category.<br>
&nbsp;&nbsp;&nbsp;$BB_9 \in Basketballs$
2. A category is a subclass of another category.<br>
&nbsp;&nbsp;&nbsp;$Basketballs \subset Balls$
3. All members of a category have some properties.<br>
&nbsp;&nbsp;&nbsp;$(x \in Basketballs) \implies Spherical (x)$
4. Members of a category can be recognized by some properties.<br>
&nbsp;&nbsp;&nbsp;$Orange(x) \wedge Round (x) \wedge Diameter (x) = 9.5''  \wedge x \in Balls \implies x \in Basketballs$
5. A category as a whole has some properties.<br>
&nbsp;&nbsp;&nbsp;$Dogs \in DomesticatedSpecies$

Let us learn how to represent this using first order logic and the code repository. We will add the code written in each block to the Utils class at the end of the notebook so that the code can be reused. Let us create a basic domain for ontology creation that includes predicates like $Member()$ and $Subset()$. Other constants and presicates can then be later added to this domain.

In [49]:
package aima.notebooks.knowledgerepresentation;

import aima.core.logic.fol.domain.FOLDomain;

FOLDomain domain = new FOLDomain();
        domain.addPredicate("Member");
        domain.addPredicate("Subset");
        return domain.toString();

aima.core.logic.fol.domain.FOLDomain@432a556d

In [50]:
package aima.notebooks.knowledgerepresentation;

import aima.core.logic.fol.domain.*;
import aima.core.logic.fol.kb.*;

public class Utils{
    
    public static FOLDomain getOntologyBasicDomain(){
        FOLDomain domain = new FOLDomain();
        domain.addPredicate("Member");
        domain.addPredicate("Subset");
        domain.addPredicate("Disjoint");
        domain.addPredicate("ExhaustiveDecomposition");
        domain.addPredicate("Partition");
        domain.addFunction("Intersection");
        domain.addConstant("Phi");
        return domain;
    }
    
    public static FOLDomain getFiveExampleDomain(){
        FOLDomain domain = Utils.getOntologyBasicDomain();
        // Add the category basketballs to the domain
        domain.addConstant("Basketballs");
        // Add the basketball BB9 to the domain
        domain.addConstant("BB9");
        // Add the category Balls
        domain.addConstant("Balls");
        domain.addConstant("Nine");// because our knowledge doesnot include real numbers
        //Add relevant properties
        domain.addPredicate("Spherical");
        domain.addPredicate("Round");
        domain.addPredicate("Orange");
        domain.addPredicate("Diameter");
        domain.addConstant("Dogs");
        domain.addConstant("DomesticatedSpecies");
        return domain;
    }
    
    public static FOLKnowledgeBase getFiveExampleKnowledgeBase(){
        FOLKnowledgeBase kb = new FOLKnowledgeBase(Utils.getOntologyBasicDomain());

        //BB9 ∈ Basketballs
        kb.tell("Member(BB9, Basketballs)");

        // Basketballs ⊂ Balls
        kb.tell("Subset(Basketballs,Balls)");

        // ( x ∈ Basketballs ) ⇒ Spherical(x) 
        kb.tell("(Member(x,Basketballs) => Spherical(x))");

        // Orange(x) ∧ Round(x) ∧ Diameter(x) = 9.5″ ∧ x ∈ Balls ⇒ x ∈ Basketballs
        kb.tell("((((Orange(x) AND Round(x)) AND Diameter(x,Nine) AND Member(x, Balls))) => Member(x, Basketballs))");

        // Dogs ∈ DomesticatedSpecies
        kb.tell("Member(Dogs,DomesticatedSpecies)");
        
        return kb;

    }
    
    public static FOLKnowledgeBase getCategoryKnowledgeBase(FOLDomain domain){
        domain.addPredicate("Member");
        domain.addPredicate("Subset");
        domain.addPredicate("Disjoint");
        domain.addPredicate("ExhaustiveDecomposition");
        domain.addPredicate("Partition");
        domain.addFunction("Intersection");
        domain.addConstant("Phi");
        domain.addPredicate("PartOf");
        domain.addPredicate("LessThan");
        domain.addPredicate("GreaterThan");
        
        FOLKnowledgeBase kb = new FOLKnowledgeBase(domain);
        // equality axioms
        // Reflexivity Axiom
        kb.tell("x = x");
        // Symmetry Axiom
        kb.tell("(x = y => y = x)");
        // Transitivity Axiom
        kb.tell("((x = y AND y = z) => x = z)");
        // Function Intersection Substitution Axiom
        kb.tell("(( x = y AND w = z) => ( Intersection(x,w) = Intersection(w,z) ))");
        // Predicate Substitution Axioms
        kb.tell("((x = y AND v = w AND Member(x,v)) => Member(y,w))");
        kb.tell("((x = y AND v = w AND Disjoint(x,v)) => Disjoint(y,w))");
        kb.tell("((x = y AND v = w AND ExhaustiveDecomposition(x,v)) => ExhaustiveDecomposition(y,w))");
        kb.tell("((x = y AND v = w AND Partition(x,v)) => Partition(y,w))");

        // Definition of disjoint Disjoint(s) ⟺ ( ∀ c1,c2 c1 ∈ s ∧ c2 ∈ s  ∧ c1 ≠ c2 ⟹ Intersection(c1,c2) = {})
        kb.tell(" ( Disjoint(s) <=> (FORALL x,y (( Member(x,s) AND Member(y,s) AND (NOT ( x = y ))) => (Intersection(x,y) = Phi)))) ");

        // Definition of ExhaustiveDecomposition ExhaustiveDecomposition(s,c) ⟺ ( ∀i i ∈ c ⟺ ∃ c2 c2 ∈ s ∧ i ∈ c2 )
        kb.tell("(ExhaustiveDecomposition(s,c) <=> ( FORALL i ( Member(i,c) => (EXISTS c2 (Member(c2,s) AND Member(i,c2))))))");

        // Definition of Partition
        kb.tell("(Partition(s,c) <=> ( Disjoint(s) AND ExhaustiveDecomposition(s)))");

        // Definition for intersection
        kb.tell("( Member(x, Intersection(s,b)) <=> (Member(x,a) AND Member(x,b)))");
    
        //Definition of Phi
        kb.tell("(FORALL x  NOT Member(x,Phi))");
        
        kb.tell("(Subset(a,b) <=> (FORALL x (Member(x,a) => Member(x,b))))");
        
        // now add the partof relation
        kb.tell("(FORALL x (PartOf(x,x)))");
        kb.tell("((PartOf(x,y) AND PartOf(y,z)) => PartOf(x,z))");
        
        //ordering axioms
        kb.tell("((LessThan(x,y) AND LessThan(y,z)) => LessThan(x,z))");
        kb.tell("((GreaterThan(x,y) AND GreaterThan(y,z)) => GreaterThan(x,z))");
        kb.tell("(LessThan(x,y) <=> GreaterThan(y,x))");
        
        return kb;
    }
}


aima.notebooks.knowledgerepresentation.Utils

Now, let's see how can we incorporate the above five facts into our ontology or knowledge base.

In [51]:

import aima.notebooks.knowledgerepresentation.Utils;
import aima.core.logic.fol.domain.*;
import aima.core.logic.fol.kb.*;

FOLDomain domain = Utils.getOntologyBasicDomain();

// Add the category basketballs to the domain
domain.addConstant("Basketballs");
// Add the basketball BB9 to the domain
domain.addConstant("BB9");
// Add the category Balls
domain.addConstant("Balls");
domain.addConstant("Nine");// because our knowledge doesnot include real numbers
//Add relevant properties
domain.addPredicate("Spherical");
domain.addPredicate("Round");
domain.addPredicate("Orange");
domain.addPredicate("Diameter");
domain.addConstant("Dogs");
domain.addConstant("DomesticatedSpecies");
// Create a knowledgebase
FOLKnowledgeBase kb = new FOLKnowledgeBase(domain);

//BB9 ∈ Basketballs
kb.tell("Member(BB9, Basketballs)");

// Basketballs ⊂ Balls
kb.tell("Subset(Basketballs,Balls)");

// ( x ∈ Basketballs ) ⇒ Spherical(x) 
kb.tell("(Member(x,Basketballs) => Spherical(x))");

// Orange(x) ∧ Round(x) ∧ Diameter(x) = 9.5″ ∧ x ∈ Balls ⇒ x ∈ Basketballs
kb.tell("((((Orange(x) AND Round(x)) AND Diameter(x,Nine) AND Member(x, Balls))) => Member(x, Basketballs))");

// Dogs ∈ DomesticatedSpecies
kb.tell("Member(Dogs,DomesticatedSpecies)");

return kb.toString();

Member(BB9,Basketballs)
Subset(Basketballs,Balls)
(Member(x,Basketballs) => Spherical(x))
((((Orange(x) AND Round(x)) AND Diameter(x,Nine)) AND Member(x,Balls)) => Member(x,Basketballs))
Member(Dogs,DomesticatedSpecies)


Now, as explained in the text we also require some other relations(such as being **disjoint**) for the categories. And even if we know that males and females
are disjoint, we will not know that an animal that is not a male must be a female, unless
we say that males and females constitute an exhaustive decomposition of the animals. A
disjoint exhaustive decomposition is known as a partition. The following examples illustrate
these three concepts:
$$
Disjoint(\{Animals, Vegetables\}) $$ $$
ExhaustiveDecomposition (\{Americans, Canadians , Mexicans\},
NorthAmericans)$$ $$
Partition(\{Males, Females\}, Animals) $$

Now, as far as first order logic is concerned the above three predicates can be defined as follows :
$$ Disjoint(s) \iff (\forall c_1 , c_2\quad c_1 \in s \wedge c_2 \in s \wedge c_1 \neq c_2 \implies Intersection(c_1 , c_2 ) = \{ \})$$
$$ ExhaustiveDecomposition (s, c) \iff (\forall i\quad i \in c \iff \exists c_2 \quad c_2 \in s \wedge i \in c_2 )$$
$$ Partition(s, c) \iff Disjoint(s) \wedge ExhaustiveDecomposition (s, c) $$

In order to add these definitions we will have to expand our basic ontology domain to include a few more constants and predicates. The Intersection must be added as a function. Also we must add a constant called `Phi` for the empty set $\phi$. Also we will later add the definition of intersection as follows:
$$ x \in A\cap B \iff (x\in \wedge x\in B)$$

In [52]:
package aima.notebooks.knowledgerepresentation;

import aima.core.logic.fol.domain.FOLDomain;

FOLDomain domain = new FOLDomain();
domain.addPredicate("Member");
domain.addPredicate("Subset");
domain.addPredicate("Disjoint");
domain.addPredicate("ExhaustiveDecomposition");
domain.addPredicate("Partition");
domain.addFunction("Intersection");
domain.addConstant("Phi");
return domain;

aima.core.logic.fol.domain.FOLDomain@6fd7b25f

Now let us create a knowledge base that incorporates the knowledge about categories and their partitions. We will call this categoryKb and we will improve it as we go ahead.

In [53]:
package aima.notebooks.knowledgerepresentation;

import aima.core.logic.fol.domain.*;
import aima.core.logic.fol.kb.*;

FOLDomain domain = Utils.getOntologyBasicDomain();
FOLKnowledgeBase kb = new FOLKnowledgeBase(domain);
// equality axioms
// Reflexivity Axiom
kb.tell("x = x");
// Symmetry Axiom
kb.tell("(x = y => y = x)");
// Transitivity Axiom
kb.tell("((x = y AND y = z) => x = z)");
// Function Intersection Substitution Axiom
kb.tell("(( x = y AND w = z) => ( Intersection(x,w) = Intersection(w,z) ))");
// Predicate Substitution Axioms
kb.tell("((x = y AND v = w AND Member(x,v)) => Member(y,w))");
kb.tell("((x = y AND v = w AND Disjoint(x,v)) => Disjoint(y,w))");
kb.tell("((x = y AND v = w AND ExhaustiveDecomposition(x,v)) => ExhaustiveDecomposition(y,w))");
kb.tell("((x = y AND v = w AND Partition(x,v)) => Partition(y,w))");

// Definition of disjoint Disjoint(s) ⟺ ( ∀ c1,c2 c1 ∈ s ∧ c2 ∈ s  ∧ c1 ≠ c2 ⟹ Intersection(c1,c2) = {})
kb.tell(" ( Disjoint(s) <=> (FORALL x,y (( Member(x,s) AND Member(y,s) AND (NOT ( x = y ))) => (Intersection(x,y) = Phi)))) ");

// Definition of ExhaustiveDecomposition ExhaustiveDecomposition(s,c) ⟺ ( ∀i i ∈ c ⟺ ∃ c2 c2 ∈ s ∧ i ∈ c2 )
kb.tell("(ExhaustiveDecomposition(s,c) <=> ( FORALL i ( Member(i,c) => (EXISTS c2 (Member(c2,s) AND Member(i,c2))))))");

// Definition of Partition
kb.tell("(Partition(s,c) <=> ( Disjoint(s) AND ExhaustiveDecomposition(s)))");

// Definition for intersection
kb.tell("( Member(x, Intersection(s,b)) <=> (Member(x,a) AND Member(x,b)))");

//Definition of Phi
kb.tell("(FORALL x  NOT Member(x,Phi))");

kb.tell("(Subset(a,b) <=> (FORALL x (Member(x,a) => Member(x,b))))");

return kb;

x = x
(x = y => y = x)
((x = y AND y = z) => x = z)
((x = y AND w = z) => Intersection(x,w) = Intersection(w,z))
(((x = y AND v = w) AND Member(x,v)) => Member(y,w))
(((x = y AND v = w) AND Disjoint(x,v)) => Disjoint(y,w))
(((x = y AND v = w) AND ExhaustiveDecomposition(x,v)) => ExhaustiveDecomposition(y,w))
(((x = y AND v = w) AND Partition(x,v)) => Partition(y,w))
(Disjoint(s) <=> FORALL x y (((Member(x,s) AND Member(y,s)) AND NOT(x = y)) => Intersection(x,y) = Phi))
(ExhaustiveDecomposition(s,c) <=> FORALL i (Member(i,c) => EXISTS c2 (Member(c2,s) AND Member(i,c2))))
(Partition(s,c) <=> (Disjoint(s) AND ExhaustiveDecomposition(s)))
(Member(x,Intersection(s,b)) <=> (Member(x,a) AND Member(x,b)))
FORALL x NOT(Member(x,Phi))
(Subset(a,b) <=> FORALL x (Member(x,a) => Member(x,b)))


Now, let's ask our knowledge base whether **GermanShepherd** is an **animal** by telling it that **GermanShepherd is a Dog** and that **all dogs are animals**. 

In [54]:
import aima.notebooks.knowledgerepresentation.Utils;
import aima.core.logic.fol.domain.*;
import aima.core.logic.fol.kb.*;

FOLDomain domain = new FOLDomain();
domain.addConstant("GermanShepherd");
domain.addConstant("Dog");
domain.addConstant("Animals");
FOLKnowledgeBase kb = Utils.getCategoryKnowledgeBase(domain);
kb.tell("Member(GermanShepherd,Dog)");
kb.tell("Subset(Dog,Animals)");
System.out.println(! kb.ask("Member(GermanShepherd,Animals)").getProofs().isEmpty());
System.out.println(! kb.ask("Member(Dog,Animals)").getProofs().isEmpty());

true
false


null

For future sections we will need this knowledge base with an extended domain. For this purpose, we have created some helper functions in the Utils class at the end  of the notebook.

## Physical composition

To specify the composition of a particular object we need the **PartOf** hierarchy. This is similar to the subset hierarchy in a few aspects. The PartOf relation is transitive and reflexive. So, we have the following relations to add to our knowledge base.
$$ PartOf (x, y) \wedge PartOf (y, z) \implies PartOf (x, z) $$
$$ PartOf (x, x) $$

We will use the following statements as our examples and then ask the knowledge base *if Bucharest is a part of Earth**.
$$PartOf (Bucharest , Romania )$$
$$PartOf (Romania, EasternEurope )$$
$$PartOf (EasternEurope , Europe)$$
$$PartOf (Europe, Earth)$$

In [55]:
package aima.notebooks.knowledgerepresentation;

import aima.core.logic.fol.domain.*;
import aima.core.logic.fol.kb.*;

FOLDomain domain = new FOLDomain();
domain.addConstant("Bucharest");
domain.addConstant("Romania");
domain.addConstant("EasternEurope");
domain.addConstant("Europe");
domain.addConstant("Earth");
domain.addPredicate("PartOf");

FOLKnowledgeBase kb = Utils.getCategoryKnowledgeBase(domain);
// now add the partof relation
kb.tell("(FORALL x (PartOf(x,x)))");
kb.tell("((PartOf(x,y) AND PartOf(y,z)) => PartOf(x,z))");

// Now add the domain specific knowledge
kb.tell("PartOf(Bucharest,Romania)");
kb.tell("PartOf(Romania,EasternEurope)");
kb.tell("PartOf(EasternEurope,Europe)");
kb.tell("PartOf(Europe,Earth)");

return kb.ask("PartOf(Bucharest,Earth)");


isComplete=true
result=[{}]

We motivate you to try the **Biped(a)** and **BunchOf(s)** relations from the book using the APIs from the code.

## Measurements

Measurements play an important role in understanding the world around us. They are an important dimension in the description of the world. To represent measurements, we can assume that the universe includes abstract measure objects such as "Length" (maybe the entire set of physical quantities or just the seven fundamental ones). Then, we can have unit functions that take numbers as arguements. We can easily represent the following sentences in first order logic by a process similar to the one demonstrated above.
$$ Diameter (Basketball 12 ) = Inches(9.5) $$
$$ ListPrice(Basketball 12 ) = $(19) $$
$$ d ∈ Days ⇒ Duration(d) = Hours(24) $$

The most important thing about measures is not the particular numerical values but the fact that the measures can be ordered. We will code the following example which demonstrates the necessity of ordering for measures quantities. Till now we have not added $LessThan$ and $GreaterThan$ predicates. Let's add them. They are exactly similar to the $PartOf$ predicate.
$$e 1 ∈ Exercises ∧ e 2 ∈ Exercises ∧ Wrote(Norvig, e 1 ) ∧ Wrote(Russell , e 2 ) ⇒
Difficulty(e 1 ) > Difficulty(e 2 ) $$
$$e 1 ∈ Exercises ∧ e 2 ∈ Exercises ∧ Difficulty(e 1 ) > Difficulty(e 2 ) ⇒
ExpectedScore (e 1 ) < ExpectedScore (e 2 )$$

In [61]:
package aima.notebooks.knowledgerepresentation;

import aima.core.logic.fol.domain.*;
import aima.core.logic.fol.kb.*;

FOLDomain domain = new FOLDomain();
domain.addConstant("Exercises");
domain.addConstant("Norvig");
domain.addConstant("Russell");
domain.addFunction("Difficulty");
domain.addFunction("ExpectedScore");
domain.addPredicate("Wrote");
domain.addPredicate("LessThan");
domain.addPredicate("GreaterThan");

FOLKnowledgeBase kb = Utils.getCategoryKnowledgeBase(domain);

// Now add the domain specific knowledge
kb.tell("((Member(e1,Exercises) AND Member(e2, Exercises) AND Wrote(Norvig,e1) AND Wrote(Russell,e2)) => (GreaterThan(Difficulty(e1), Difficulty(e2))))");

return kb;

x = x
(x = y => y = x)
((x = y AND y = z) => x = z)
((x = y AND w = z) => Intersection(x,w) = Intersection(w,z))
(((x = y AND v = w) AND Member(x,v)) => Member(y,w))
(((x = y AND v = w) AND Disjoint(x,v)) => Disjoint(y,w))
(((x = y AND v = w) AND ExhaustiveDecomposition(x,v)) => ExhaustiveDecomposition(y,w))
(((x = y AND v = w) AND Partition(x,v)) => Partition(y,w))
(Disjoint(s) <=> FORALL x y (((Member(x,s) AND Member(y,s)) AND NOT(x = y)) => Intersection(x,y) = Phi))
(ExhaustiveDecomposition(s,c) <=> FORALL i (Member(i,c) => EXISTS c2 (Member(c2,s) AND Member(i,c2))))
(Partition(s,c) <=> (Disjoint(s) AND ExhaustiveDecomposition(s)))
(Member(x,Intersection(s,b)) <=> (Member(x,a) AND Member(x,b)))
FORALL x NOT(Member(x,Phi))
(Subset(a,b) <=> FORALL x (Member(x,a) => Member(x,b)))
FORALL x PartOf(x,x)
((PartOf(x,y) AND PartOf(y,z)) => PartOf(x,z))
((LessThan(x,y) AND LessThan(y,z)) => LessThan(x,z))
((GreaterThan(x,y) AND GreaterThan(y,z)) => GreaterThan(x,z))
(LessThan(x,y) <=> Greater

Try adding the second condition and experimenting with the knowledge base.

## Objects: Things and stuff